In [1]:
# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
import sys; sys.path.append('./stepik-dl-nlp')

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as F

import sys
import os
from tqdm import tqdm  
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim import utils
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer



# import dlnlputils
# from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
#     vectorize_texts, SparseFeaturesDataset
# from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

# init_random_seed()

In [2]:
import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset

In [3]:
import collections
import re

import numpy as np

TOKEN_RE = re.compile(r'[\w\d]+')


def tokenize_text_simple_regex(txt, min_token_size=4):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]


def character_tokenize(txt):
    return list(txt)


def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]


def add_fake_token(word2id, token=''):
    word2id_new = {token: i + 1 for token, i in word2id.items()}
    word2id_new[token] = 0
    return word2id_new


def texts_to_token_ids(tokenized_texts, word2id):
    return [[word2id[token] for token in text if token in word2id]
            for text in tokenized_texts]


def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq


PAD_TOKEN = '__PAD__'
NUMERIC_TOKEN = '__NUMBER__'
NUMERIC_RE = re.compile(r'^([0-9.,e+\-]+|[mcxvi]+)$', re.I)


def replace_number_nokens(tokenized_texts):
    return [[token if not NUMERIC_RE.match(token) else NUMERIC_TOKEN for token in text]
            for text in tokenized_texts]

In [4]:
import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset


def vectorize_texts(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
    assert mode in {'tfidf', 'idf', 'tf', 'bin'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(1 / word2freq)  # разделить каждый столбец на вес слова

    if scale:
        result = result.tocsc()
        result -= result.min()
        result /= (result.max() + 1e-6)

    return result.tocsr()


class SparseFeaturesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        cur_features = torch.from_numpy(self.features[idx].toarray()[0]).float()
        cur_label = torch.from_numpy(np.asarray(self.targets[idx])).long()
        return cur_features, cur_label

In [5]:
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [195]:
import gensim.downloader as api
for k, v in api.info().items():
    for k1, v1 in v.items():
        print(k1)

semeval-2016-2017-task3-subtaskBC
semeval-2016-2017-task3-subtaskA-unannotated
patent-2017
quora-duplicate-questions
wiki-english-20171001
text8
fake-news
20-newsgroups
__testing_matrix-synopsis
__testing_multipart-matrix-synopsis
fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [64]:
from navec import Navec
from gensim.models import Word2Vec

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [43]:
model_t = api.load('glove-twitter-100')
model_t1 = api.load('glove-twitter-200')
model_t2 = api.load('glove-wiki-gigaword-200')

In [69]:
from gensim import utils

def get_vectors_gt100(row):
    '''
      word_doc_freq # частоты слов
      train_tokenized #сами слова
    '''
#     vecs = [np.zeros(100)]
    vecs = [np.zeros(300)]

    for word in row:
        #print(row)
        try: 
            # если слово есть в нашем очищенном словаре
            # умножаем вектор на вес tfidf
            v = navec[word] * word_doc_freq[vocabulary[word]] 
#             v = model_t[word] * word_doc_freq[vocabulary[word]] 

        except:
#             v = np.zeros(100)
            v = np.zeros(300)
        vecs.append(v)
    return np.sum(np.array(vecs),axis=0)

In [42]:
from gensim import utils

def get_vectors_gt100(row):
    '''
      word_doc_freq # частоты слов
      train_tokenized #сами слова
    '''
    vecs = [np.zeros(100)]
    for word in row:
        #print(row)
        try: 
            # если слово есть в нашем очищенном словаре
            # умножаем вектор на вес tfidf
            v = model_t[word] * word_doc_freq[vocabulary[word]] 
        except:
            v = np.zeros(100)
        vecs.append(v)
    return np.sum(np.array(vecs),axis=0)

In [8]:
input_path = 'C:\\Users\\eduard\\jup\\project\\данные' # нужно указать путь до папки с данными по новостям

for root, dirs, files in os.walk(input_path):
    for q in tqdm(range(1, 12)):
        df = pd.DataFrame([])
        for file in files[:q]:
            df1 = pd.read_csv(f'данные/{file}')
            df = pd.concat([df, df1], axis=0, ignore_index=True)

        df = df.sort_values('date', ignore_index=True)
        df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

        pnl = pd.read_csv('данные/zBacktest_EURUSD_5_days.txt')
        pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
        pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
        pnl['cumm_text'] = None

        df_with_pnl = df[(df['date'] >= '2022-03-02') & (df['date'] <= '2022-11-01')]

        i = 0
        texts = ''
        for _,row in df_with_pnl.iterrows():
            if i >= pnl.shape[0]:
                break

            if row['date'] <= pnl['date_start'][i]:
                texts += ' ' + row['text']
            else:
                pnl.at[i,'cumm_text'] = texts
                texts = ''
                i += 1
                continue

        train, val = train_test_split(pnl)
        train_tokenized = tokenize_corpus(train['cumm_text'])
        val_tokenized = tokenize_corpus(val['cumm_text'])

        MAX_DF = 0.8 #во скольких документах встречаеться слово
        MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


        vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
        UNIQUE_WORDS_N = len(vocabulary)    
        VECTORIZATION_MODE = 'tfidf'
        train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

        train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
        val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

        model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

        y_train = train['pnl_sign']
        X_train = train_vectors_gt100

        y_val = val['pnl_sign']
        X_val = val_vectors_gt100

        model = LogisticRegression()
        model.fit(X_train, y_train)
        print(model.score(X_val, y_val))
        print('данные:', files[:q])
        print('всего наблюдений:', df_with_pnl.shape[0])


  9%|███████▌                                                                           | 1/11 [00:00<00:05,  1.86it/s]

0.4166666666666667
данные: ['Alfa_Wealth.csv']
всего наблюдений: 199


 18%|███████████████                                                                    | 2/11 [00:08<00:46,  5.15s/it]

0.375
данные: ['Alfa_Wealth.csv', 'bitkogan.csv']
всего наблюдений: 2315


 27%|██████████████████████▋                                                            | 3/11 [00:21<01:08,  8.52s/it]

...

0.5416666666666666
данные: ['Alfa_Wealth.csv', 'bitkogan.csv', 'cbonds.csv', 'headlines_QUANTS.csv', 'mmi.csv', 'rshb_invest.csv', 'signal.csv', 'sky_bond.csv', 'themovchans.csv']
всего наблюдений: 16573


 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [02:37<00:20, 20.81s/it]

0.625
данные: ['Alfa_Wealth.csv', 'bitkogan.csv', 'cbonds.csv', 'headlines_QUANTS.csv', 'mmi.csv', 'rshb_invest.csv', 'signal.csv', 'sky_bond.csv', 'themovchans.csv', 'vts.csv']
всего наблюдений: 16735


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [03:02<00:00, 16.60s/it]

0.5
данные: ['Alfa_Wealth.csv', 'bitkogan.csv', 'cbonds.csv', 'headlines_QUANTS.csv', 'mmi.csv', 'rshb_invest.csv', 'signal.csv', 'sky_bond.csv', 'themovchans.csv', 'vts.csv', 'War_Wealth_Wisdom.csv']
всего наблюдений: 17025


In [222]:
input_path = 'C:\\Users\\eduard\\jup\\project\\данные'

for root, dirs, files in os.walk(input_path):
    for q in tqdm(range(1, 12)):
        df = pd.DataFrame([])
        for file in files[:q]:
            df1 = pd.read_csv(f'данные/{file}')
            df = pd.concat([df, df1], axis=0, ignore_index=True)

        df = df.sort_values('date', ignore_index=True)
        df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

        pnl = pd.read_csv('данные/zBacktest_EURUSD_5_days.txt')
        pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
        pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
        pnl['cumm_text'] = None

        df_with_pnl = df[(df['date'] >= '2022-03-02') & (df['date'] <= '2022-11-01')]

        i = 0
        texts = ''
        for _,row in tqdm(df_with_pnl.iterrows()):
            if i >= pnl.shape[0]:
                break

            if row['date'] <= pnl['date_start'][i]:
                texts += ' ' + row['text']
            else:
                pnl.at[i,'cumm_text'] = texts
                texts = ''
                i += 1
                continue

        train, val = train_test_split(pnl)
        train_tokenized = tokenize_corpus(train['cumm_text'])
        val_tokenized = tokenize_corpus(val['cumm_text'])

        MAX_DF = 0.8 #во скольких документах встречаеться слово
        MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


        vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
        UNIQUE_WORDS_N = len(vocabulary)    
        VECTORIZATION_MODE = 'tfidf'
        train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

        train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
        val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

        model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

        y_train = train['pnl_sign']
        X_train = train_vectors_gt100

        y_val = val['pnl_sign']
        X_val = val_vectors_gt100

        model = LogisticRegression()
        model.fit(X_train, y_train)
        print(model.score(X_val, y_val))
        print('данные:', files[:q])
        print('всего наблюдений:', df_with_pnl.shape[0])


  0%|                                                                                           | 0/11 [00:00<?, ?it/s]
191it [00:00, 10402.43it/s]
  9%|███████▌                                                                           | 1/11 [00:00<00:05,  1.86it/s]

0.5
данные: ['Alfa_Wealth.csv']
всего наблюдений: 199



0it [00:00, ?it/s]
629it [00:00, 6080.93it/s]
1238it [00:00, 3290.57it/s]
2236it [00:00, 4307.34it/s]
 18%|███████████████                                                                    | 2/11 [00:07<00:36,  4.10s/it]

0.625
данные: ['Alfa_Wealth.csv', 'bitkogan.csv']
всего наблюдений: 2315



0it [00:00, ?it/s]
962it [00:00, 9053.32it/s]
1868it [00:00, 2301.28it/s]
2913it [00:00, 3683.99it/s]
4269it [00:01, 4095.89it/s]
 27%|██████████████████████▋                                                            | 3/11 [00:18<00:59,  7.44s/it]

...

0.5416666666666666
данные: ['Alfa_Wealth.csv', 'bitkogan.csv', 'cbonds.csv', 'headlines_QUANTS.csv', 'mmi.csv', 'rshb_invest.csv', 'signal.csv', 'sky_bond.csv', 'themovchans.csv']
всего наблюдений: 16573



0it [00:00, ?it/s]
1515it [00:00, 14522.98it/s]
3148it [00:00, 15082.02it/s]
4656it [00:00, 5207.23it/s] 
5607it [00:01, 3040.64it/s]
6241it [00:01, 2200.24it/s]
6686it [00:02, 1793.12it/s]
7013it [00:02, 1503.42it/s]
7259it [00:02, 1527.27it/s]
8647it [00:03, 2918.11it/s]
10159it [00:03, 4602.25it/s]
11017it [00:03, 4284.21it/s]
11723it [00:03, 3367.11it/s]
12275it [00:04, 2964.29it/s]
13761it [00:04, 4586.97it/s]
16119it [00:04, 3757.14it/s]
 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [02:34<00:20, 20.97s/it]

0.5416666666666666
данные: ['Alfa_Wealth.csv', 'bitkogan.csv', 'cbonds.csv', 'headlines_QUANTS.csv', 'mmi.csv', 'rshb_invest.csv', 'signal.csv', 'sky_bond.csv', 'themovchans.csv', 'vts.csv']
всего наблюдений: 16735



0it [00:00, ?it/s]
1270it [00:00, 11457.90it/s]
2771it [00:00, 13277.52it/s]
4105it [00:00, 7098.52it/s] 
5035it [00:01, 3818.33it/s]
5672it [00:01, 2699.38it/s]
6129it [00:01, 2045.11it/s]
6465it [00:02, 1694.43it/s]
6721it [00:02, 1507.97it/s]
6926it [00:02, 1309.79it/s]
7090it [00:03, 1168.42it/s]
7226it [00:03, 1069.13it/s]
7777it [00:03, 1693.05it/s]
9219it [00:03, 3832.40it/s]
10531it [00:03, 5612.52it/s]
11344it [00:03, 4455.36it/s]
12000it [00:04, 3187.70it/s]
12510it [00:04, 2822.40it/s]
13847it [00:04, 4354.81it/s]
16401it [00:04, 3453.40it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [02:54<00:00, 15.83s/it]

0.5
данные: ['Alfa_Wealth.csv', 'bitkogan.csv', 'cbonds.csv', 'headlines_QUANTS.csv', 'mmi.csv', 'rshb_invest.csv', 'signal.csv', 'sky_bond.csv', 'themovchans.csv', 'vts.csv', 'War_Wealth_Wisdom.csv']
всего наблюдений: 17025


# Новый

In [102]:
input_path = 'C:\\Users\\eduard\\jup\\project\\данные'

df = pd.DataFrame([])
for root, dirs, files in os.walk(input_path):
    for file in tqdm(files[:5]):
        df1 = pd.read_csv(f'данные\{file}')
        df = pd.concat([df, df1], axis=0, ignore_index=True)

df = df.sort_values('date', ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.19it/s]


In [103]:
pnl = pd.read_csv('данные/zBacktest_USDRUB_5_days.txt')
pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
pnl.sort_values('date_start')

,date_start,pnl,pnl_sign
0,2021-01-04,291264.097914,1
1,2021-01-05,-411993.830320,0
2,2021-01-06,521491.686795,1
3,2021-01-08,62842.634116,1
4,2021-01-11,-537598.706217,0
...,...,...,...
245,2021-12-20,-407527.554561,0
246,2021-12-21,-176881.417077,0
247,2021-12-22,-206943.414418,0
248,2021-12-23,23073.596468,1


In [104]:
pnl['cumm_text'] = None
pnl

,date_start,pnl,pnl_sign,cumm_text
0,2021-01-04,291264.097914,1,None
1,2021-01-05,-411993.830320,0,None
2,2021-01-06,521491.686795,1,None
3,2021-01-08,62842.634116,1,None
4,2021-01-11,-537598.706217,0,None
...,...,...,...,...
245,2021-12-20,-407527.554561,0,None
246,2021-12-21,-176881.417077,0,None
247,2021-12-22,-206943.414418,0,None
248,2021-12-23,23073.596468,1,None


In [105]:
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

In [106]:
df_with_pnl = df[(df['date'] >= '2021-01-04') & (df['date'] <= '2021-12-24')]

In [107]:
pnl['cumm_text'] = df_with_pnl.groupby('date', as_index = False).agg({'text': ' '.join})['text']
pnl

,date_start,pnl,pnl_sign,cumm_text
0,2021-01-04,291264.097914,1,🟢 - настроения risk-on доминируют в начале нас...
1,2021-01-05,-411993.830320,0,🔴 - умеренно-негативные настроения с утра: рос...
2,2021-01-06,521491.686795,1,"ВСЕМИРНЫЙ БАНК ПОВЫСИЛ ПРОГНОЗЫ, КОТОРЫЕ, ТЕМ ..."
3,2021-01-08,62842.634116,1,НОВЫЙ РЕКОРД ПО ЧИСЛУ ЗАРАЗИВШИХСЯ В МИРЕ За п...
4,2021-01-11,-537598.706217,0,РУБЛЕВЫЙ ИНДЕКС МИРОВЫХ ЦЕН НА ПРОДОВОЛЬСТВИЕ ...
...,...,...,...,...
245,2021-12-20,-407527.554561,0,Что ожидать инвестору от предстоящей недели. Д...
246,2021-12-21,-176881.417077,0,"РЫНОК УВЕРЕН, ЧТО ВЫШЕ 7% КЛЮЧЕВАЯ СТАВКА НЕ П..."
247,2021-12-22,-206943.414418,0,• Обвал криптовалют не привёл к сколь-либо за...
248,2021-12-23,23073.596468,1,• Над Бразилией продолжают сгущаться тучи ...


In [108]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(pnl)

tqdm.pandas()

train['lemmas'] = train['cumm_text'].progress_apply(lambda x: clean_text(x))
val['lemmas'] = val['cumm_text'].progress_apply(lambda x: clean_text(x))

100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [00:32<00:00,  1.92it/s]


In [116]:
ct = ColumnTransformer([
    ('vec', TfidfVectorizer(ngram_range=(1, 2), max_df = 0.95, min_df = 0.001), 'lemmas')
    ])

train_tr = ct.fit_transform(train)
clf = LogisticRegression(random_state=42, max_iter=500, warm_start=True)

y_train = train['pnl_sign'].values
clf.fit(train_tr, y_train)

test_data = ct.transform(val)
pred = clf.predict(test_data)

In [47]:
train_tokenized = tokenize_corpus(train['lemmas'])
val_tokenized = tokenize_corpus(val['lemmas'])

MAX_DF = 0.8 #во скольких документах встречаеться слово
MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)    
VECTORIZATION_MODE = 'tfidf'
train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

y_train = train['pnl_sign']
X_train = train_vectors_gt100

y_val = val['pnl_sign']
X_val = val_vectors_gt100

model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_val, y_val))

0.5873015873015873


In [109]:
# train_tokenized = tokenize_corpus(train['lemmas'])
# val_tokenized = tokenize_corpus(val['lemmas'])
train_tokenized = tokenize_corpus(train['cumm_text'])
val_tokenized = tokenize_corpus(val['cumm_text'])

MAX_DF = 0.8 #во скольких документах встречаеться слово
MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)    
VECTORIZATION_MODE = 'tfidf'
train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

y_train = train['pnl_sign']
X_train = train_vectors_gt100

y_val = val['pnl_sign']
X_val = val_vectors_gt100

model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_val, y_val))

0.5238095238095238


In [110]:
train_tokenized = tokenize_corpus(train['lemmas'])
val_tokenized = tokenize_corpus(val['lemmas'])
# train_tokenized = tokenize_corpus(train['cumm_text'])
# val_tokenized = tokenize_corpus(val['cumm_text'])

MAX_DF = 0.8 #во скольких документах встречаеться слово
MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)    
VECTORIZATION_MODE = 'tfidf'
train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

y_train = train['pnl_sign']
X_train = train_vectors_gt100

y_val = val['pnl_sign']
X_val = val_vectors_gt100

model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_val, y_val))

0.47619047619047616


In [44]:
train_tokenized = tokenize_corpus(train['cumm_text'])
val_tokenized = tokenize_corpus(val['cumm_text'])

MAX_DF = 0.8 #во скольких документах встречаеться слово
MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)    
VECTORIZATION_MODE = 'tfidf'
train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

y_train = train['pnl_sign']
X_train = train_vectors_gt100

y_val = val['pnl_sign']
X_val = val_vectors_gt100

model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_val, y_val))

0.4583333333333333
